In [1]:
with open('tarzan.txt', encoding="utf8") as f:
    corpus = f.read().replace('\n',' ')

<h2> Preprocessing </h2>

In [2]:
import re
corpus = corpus.lower()

normalized_text = re.sub(r'[^a-zA-Z0-9\s]', '', corpus)

<h2> Tokenization </h2>

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

tokenized_text = word_tokenize(normalized_text)

#word_tokens = word_tokenize(normalized_text)
#tokenized_text = []
 
#for w in word_tokens:
#    if w not in stop_words:
#        tokenized_text.append(w)


<h2> Bigram </h2>

In [4]:
from collections import Counter
import itertools

# Finding Bigrams
bigrams = nltk.ngrams(tokenized_text, 2)
bigrams_count = Counter(bigrams)
words_count = Counter(tokenized_text)

uniqueWords = [] 
for i in tokenized_text:
      if not i in uniqueWords:
        uniqueWords.append(i);

all_bigrams = list(itertools.product(uniqueWords, uniqueWords))
len(all_bigrams)

46881409

In [5]:
bigrams_count

Counter({('the', 'project'): 33,
         ('project', 'gutenberg'): 87,
         ('gutenberg', 'ebook'): 3,
         ('ebook', 'of'): 1,
         ('of', 'tarzan'): 26,
         ('tarzan', 'lord'): 10,
         ('lord', 'of'): 12,
         ('of', 'the'): 966,
         ('the', 'jungle'): 95,
         ('jungle', 'this'): 1,
         ('this', 'ebook'): 8,
         ('ebook', 'is'): 2,
         ('is', 'for'): 3,
         ('for', 'the'): 96,
         ('the', 'use'): 6,
         ('use', 'of'): 10,
         ('of', 'anyone'): 2,
         ('anyone', 'anywhere'): 2,
         ('anywhere', 'in'): 2,
         ('in', 'the'): 372,
         ('the', 'united'): 15,
         ('united', 'states'): 16,
         ('states', 'and'): 3,
         ('and', 'most'): 3,
         ('most', 'other'): 2,
         ('other', 'parts'): 2,
         ('parts', 'of'): 3,
         ('the', 'world'): 15,
         ('world', 'at'): 2,
         ('at', 'no'): 4,
         ('no', 'cost'): 2,
         ('cost', 'and'): 2,
         ('and',

<b> Calculate Probabilities <b>

In [6]:
bigram_probabilities = {}

for bigram in all_bigrams:
    word = bigram[0]
    bigram_probabilities[bigram] = bigrams_count[bigram] / words_count[word]

In [7]:
zeros=0
for i in bigram_probabilities:
    if(bigram_probabilities[i]==0):
        zeros+=1
zeros

46841856

<h3> Smoothed Probabilities </h3>

In [8]:
add1_probabilities = {}
V = len(uniqueWords)
for bigram in all_bigrams:
        word = bigram[0]
        alpha = 1
        add1_probabilities[bigram] = ( bigrams_count[bigram] + alpha) / (words_count[word] + (alpha*V))
        
add1_probabilities

{('the', 'the'): 8.061915511125443e-05,
 ('the', 'project'): 0.0027410512737826506,
 ('the', 'gutenberg'): 8.061915511125443e-05,
 ('the', 'ebook'): 8.061915511125443e-05,
 ('the', 'of'): 8.061915511125443e-05,
 ('the', 'tarzan'): 8.061915511125443e-05,
 ('the', 'lord'): 0.0002418574653337633,
 ('the', 'jungle'): 0.007739438890680426,
 ('the', 'this'): 8.061915511125443e-05,
 ('the', 'is'): 8.061915511125443e-05,
 ('the', 'for'): 8.061915511125443e-05,
 ('the', 'use'): 0.000564334085778781,
 ('the', 'anyone'): 8.061915511125443e-05,
 ('the', 'anywhere'): 8.061915511125443e-05,
 ('the', 'in'): 8.061915511125443e-05,
 ('the', 'united'): 0.0012899064817800709,
 ('the', 'states'): 8.061915511125443e-05,
 ('the', 'and'): 8.061915511125443e-05,
 ('the', 'most'): 0.0007255723960012899,
 ('the', 'other'): 0.003869719445340213,
 ('the', 'parts'): 8.061915511125443e-05,
 ('the', 'world'): 0.0012899064817800709,
 ('the', 'at'): 8.061915511125443e-05,
 ('the', 'no'): 8.061915511125443e-05,
 ('the'

In [9]:
zeros=0
for i in add1_probabilities:
    if(add1_probabilities[i]==0):
        zeros+=1
zeros

0

<h2> Completing Sentence </h2>
<h3> first Approach </h3>

In [93]:
import numpy as np
sentence = "Knowing well the windings of the trail he"

for _ in range(10):
    last_word = sentence.split()[-1]
    possible_words = [word for (prev_word, word) in add1_probabilities.keys() if prev_word == last_word]
    probabilities = [add1_probabilities[(last_word, word)] for word in possible_words]
    next_word_idx = np.argmax(probabilities)
    # next_word = max(possible_words, probabilities)[0]
    next_word = possible_words[next_word_idx]
    sentence += " " + next_word

print(sentence)

Knowing well the windings of the trail he had been a great tourney and the great tourney and


In [94]:
sentence = "For half a day he lolled on the huge back and"

for _ in range(10):
    last_word = sentence.split()[-1]
    possible_words = [word for (prev_word, word) in add1_probabilities.keys() if prev_word == last_word]
    probabilities = [add1_probabilities[(last_word, word)] for word in possible_words]
    next_word_idx = np.argmax(probabilities)
    # next_word = max(possible_words, probabilities)[0]
    next_word = possible_words[next_word_idx]
    sentence += " " + next_word

print(sentence)

For half a day he lolled on the huge back and the great tourney and the great tourney and the great


<h3> Second Approach on Bigram Before Smoothing </h3>

In [99]:
import random

sentence = "Knowing well the windings of the trail he"

for _ in range(10):
    last_word = sentence.split()[-1]
    possible_words = [word for (prev_word, word) in bigram_probabilities.keys() if prev_word == last_word]
    probabilities = [bigram_probabilities[(last_word, word)] for word in possible_words]
    next_word = random.choices(possible_words, weights=probabilities)[0]
    sentence += " " + next_word

print(sentence)

Knowing well the windings of the trail he had witnessed the freedmen and whats wrong with project gutenberg


In [100]:
sentence = "For half a day he lolled on the huge back and"

for _ in range(10):
    last_word = sentence.split()[-1]
    possible_words = [word for (prev_word, word) in bigram_probabilities.keys() if prev_word == last_word]
    probabilities = [bigram_probabilities[(last_word, word)] for word in possible_words]
    next_word = random.choices(possible_words, weights=probabilities)[0]
    sentence += " " + next_word

print(sentence)

For half a day he lolled on the huge back and silver and the spokesman all the sun was to their


<h3> Second Approach on Bigram after Smoothing </h3>

In [97]:
import random

sentence = "Knowing well the windings of the trail he"

for _ in range(10):
    last_word = sentence.split()[-1]
    possible_words = [word for (prev_word, word) in add1_probabilities.keys() if prev_word == last_word]
    probabilities = [add1_probabilities[(last_word, word)] for word in possible_words]
    next_word = random.choices(possible_words, weights=probabilities)[0]
    sentence += " " + next_word

print(sentence)

Knowing well the windings of the trail he already fastened defenders offended disappear waziri axiomatic clambered urge bonds


In [101]:
sentence = "For half a day he lolled on the huge back and"

for _ in range(10):
    last_word = sentence.split()[-1]
    possible_words = [word for (prev_word, word) in add1_probabilities.keys() if prev_word == last_word]
    probabilities = [add1_probabilities[(last_word, word)] for word in possible_words]
    next_word = random.choices(possible_words, weights=probabilities)[0]
    sentence += " " + next_word

print(sentence)

For half a day he lolled on the huge back and met quotasee laborers mused pig crown harmed treasure restlessness asserted


<h2> Part 4: n-Grams </h2>

In [109]:
def create_n_gram_model(tokenized_text, n):
    bigrams = nltk.ngrams(tokenized_text, n)
    bigrams_count = Counter(bigrams)
    words_count = Counter(nltk.ngrams(tokenized_text, n-1))
    return bigrams_count, words_count

trigrams, bigrams = create_n_gram_model(tokenized_text, 3)
pentagrams, quadgrams  = create_n_gram_model(tokenized_text, 5)

In [110]:
def smoothing(n_grams, n_minus_grams, n_minus_gram):
    smoothed_n_gram = []
    smoothed_n_gram_probs = []
    for word in uniqueWords:
        smoothed_n_gram.append(n_minus_gram + (word, ))
        if n_minus_gram + (word, ) in n_grams.keys():
            temp = n_grams[smoothed_n_gram[-1]]
        else:
            temp = 0
        smoothed_n_gram_probs.append((temp + 1)/(n_minus_grams[n_minus_gram] + len(n_minus_grams)))
    return smoothed_n_gram, smoothed_n_gram_probs


<h2> Completing Sentence - 3-gram </h2>

In [111]:
sentence = "Knowing well the windings of the trail he"

for _ in range(10):
    last_word_1 = sentence.split()[-1]
    last_word_2 = sentence.split()[-2]
    possibles, probabilities = smoothing(trigrams, bigrams, (last_word_2, last_word_1))
    next_word_idx = np.argmax(probabilities)
    # next_word = max(possible_words, probabilities)[0]
    next_word = possibles[next_word_idx]
    sentence += " " + next_word[-1]

print(sentence)

Knowing well the windings of the trail he did not know that he had been the original company


In [112]:
sentence = "For half a day he lolled on the huge back and"

for _ in range(10):
    last_word_1 = sentence.split()[-1]
    last_word_2 = sentence.split()[-2]
    possibles, probabilities = smoothing(trigrams, bigrams, (last_word_2, last_word_1))
    next_word_idx = np.argmax(probabilities)
    # next_word = max(possible_words, probabilities)[0]
    next_word = possibles[next_word_idx]
    sentence += " " + next_word[-1]

print(sentence)

For half a day he lolled on the huge back and forth slowly there were no others about the lists and


<h2> Completing Sentence - 5-gram </h2>

In [118]:
sentence = "Knowing well the windings of the trail he"

for _ in range(10):
    last_word_1 = sentence.split()[-1]
    last_word_2 = sentence.split()[-2]
    last_word_3 = sentence.split()[-3]
    last_word_4 = sentence.split()[-4]
    possibles, probabilities = smoothing(pentagrams, quadgrams, (last_word_4, last_word_3, last_word_2, last_word_1))
    next_word_idx = np.argmax(probabilities)
    # next_word = max(possible_words, probabilities)[0]
    next_word = possibles[next_word_idx]
    sentence += " " + next_word[-1]

print(sentence)

Knowing well the windings of the trail he took short cuts swinging through the branches of the trees


In [124]:
sentence = "For half a day he lolled on the huge back and"

for _ in range(10):
    last_word_1 = sentence.split()[-1]
    last_word_2 = sentence.split()[-2]
    last_word_3 = sentence.split()[-3]
    last_word_4 = sentence.split()[-4]
    possibles, probabilities = smoothing(pentagrams, quadgrams, (last_word_4, last_word_3, last_word_2, last_word_1))
    next_word_idx = np.argmax(probabilities)
    # next_word = max(possible_words, probabilities)[0]
    next_word = possibles[next_word_idx]
    sentence += " " + next_word[-1]

print(sentence)

For half a day he lolled on the huge back and the the the the the the the the the the


In [125]:
(smoothing(pentagrams, quadgrams, ('the', 'huge', 'back', 'and'))[1])

[1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
 1.4226167612706813e-05,
